In [51]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from utils.data_generation import generate_vectors, generate_scalars
from benchmark_functions.sphere import sphere_func
from neural_network_models.feedforward_nn import FeedforwardNN

In [52]:
# for reproducability 
np.random.seed(1000)
torch.manual_seed(1000)

In [53]:
# training and test data settings
input_dimention = 5
train_set_size = 800
test_set_size = 200
domain = [-1, 1]

In [54]:
# generate training data
training_set_vectors = generate_vectors(input_dimention, domain, train_set_size)
training_set_scalars = generate_scalars(training_set_vectors, sphere_func)

# generate test data
test_set_vectors = generate_vectors(input_dimention, domain, test_set_size)
test_set_scalars = generate_vectors(input_dimention, domain, test_set_size)

# convert generated data to tensors
training_set_vectors = torch.FloatTensor(training_set_vectors)
training_set_scalars = torch.FloatTensor(training_set_scalars)

test_set_vectors = torch.FloatTensor(test_set_vectors)
test_set_scalars = torch.FloatTensor(test_set_scalars)

In [55]:
# create a NN model
model = FeedforwardNN(input_neuron_num = input_dimention, h1_neuron_num = 10, output_neuron_num = 1)

loss_func = nn.MSELoss()
loss_optimization_func = optim.Adam(model.parameters(), lr = 0.01)

In [56]:
# train NN model
learning_epochs = 100
for epoch in range(learning_epochs):
    # pass forward
    training_set_predictions = model(training_set_vectors)
    training_loss = loss_func(training_set_predictions, training_set_scalars)

    # back-propagation
    loss_optimization_func.zero_grad()
    training_loss.backward()
    loss_optimization_func.step()
    
    # Print progress occasionally
    if (epoch+1) % 10 == 0:
        print(f"[{epoch+1}/{learning_epochs}] Training loss: {training_loss.item():.6f}")

[10/100] Training loss: 2.902060
[20/100] Training loss: 1.355459
[30/100] Training loss: 0.638978
[40/100] Training loss: 0.475616
[50/100] Training loss: 0.489379
[60/100] Training loss: 0.487556
[70/100] Training loss: 0.475832
[80/100] Training loss: 0.473529
[90/100] Training loss: 0.473977
[100/100] Training loss: 0.473621


In [57]:
# evaluate model
model.eval()

with torch.no_grad():
    test_set_predictions = model(test_set_vectors)
    test_loss = loss_func(test_set_predictions, test_set_scalars)
    print(f"Test loss (MSE): {test_loss.item():.6f}")

Test loss (MSE): 3.145237
